In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("LogistcRegerssion").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/11 12:20:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark.ml.classification import LogisticRegression

In [12]:
my_data = spark.read.format("libsvm").load("sample_libsvm_data.txt")

24/03/11 12:25:13 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [13]:
my_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [5]:
my_log_reg_model = LogisticRegression()

In [14]:
logreg_model = my_log_reg_model.fit(my_data)

In [8]:
log_summary = logreg_model.summary

In [10]:
log_summary.predictions.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [11]:
log_summary.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[127,128,129...|[20.3777627514872...|[0.99999999858729...|       0.0|
|  1.0|(692,[158,159,160...|[-21.114014198868...|[6.76550380000472...|       1.0|
|  1.0|(692,[124,125,126...|[-23.743613234676...|[4.87842678716177...|       1.0|
|  1.0|(692,[152,153,154...|[-19.192574012720...|[4.62137287298144...|       1.0|
|  1.0|(692,[151,152,153...|[-20.125398874699...|[1.81823629113068...|       1.0|
|  0.0|(692,[129,130,131...|[20.4890549504196...|[0.99999999873608...|       0.0|
|  1.0|(692,[158,159,160...|[-21.082940212814...|[6.97903542823766...|       1.0|
|  1.0|(692,[99,100,101,...|[-19.622713503550...|[3.00582577446132...|       1.0|
|  0.0|(692,[154,155,156...|[21.1594863606582...|[0.99999999935352...|       0.0|
|  0.0|(692,[127

In [15]:
lr_train, lr_test = my_data.randomSplit([0.7, 0.3])

In [16]:
final_model = LogisticRegression()

In [17]:
fit_final = final_model.fit(lr_train)

In [18]:
prediction_and_label = fit_final.evaluate(lr_test)

In [20]:
prediction_and_label.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[122,123,148...|[8.86446114722697...|[0.99985869677721...|       0.0|
|  0.0|(692,[124,125,126...|[82.0783887890731...|           [1.0,0.0]|       0.0|
|  0.0|(692,[126,127,128...|[23.5839496719561...|[0.99999999994277...|       0.0|
|  0.0|(692,[126,127,128...|[26.5616890197984...|[0.99999999999708...|       0.0|
|  0.0|(692,[128,129,130...|[17.0887012846148...|[0.99999996211464...|       0.0|
|  0.0|(692,[151,152,153...|[25.4253914340027...|[0.99999999999092...|       0.0|
|  0.0|(692,[153,154,155...|[25.8719099835829...|[0.99999999999419...|       0.0|
|  0.0|(692,[153,154,155...|[2.00433880735802...|[0.88125187279096...|       0.0|
|  0.0|(692,[154,155,156...|[17.7612722878849...|[0.99999998066351...|       0.0|
|  0.0|(692,[155

In [21]:
from pyspark.ml.evaluation import (
    BinaryClassificationEvaluator,
    MulticlassClassificationEvaluator,
)

In [22]:
my_eval = BinaryClassificationEvaluator()

In [24]:
my_final_roc = my_eval.evaluate(prediction_and_label.predictions)

In [25]:
my_final_roc

1.0